In [1]:
BIOMARKER.DIR <- "../../data/biomarker/"
biomarker_traits <- list.files(BIOMARKER.DIR, pattern="dat_\\D*.RData")

biomarkers <- sapply(biomarker_traits, function(x) strsplit(strsplit(x, "dat_",fixed=TRUE)[[1]][[2]], ".RData")[[1]][[1]])

biomarkers2 <- setdiff(biomarkers , c("arm_fp", "leg_fp", "trunk_fp", "whr"))
biomarkers2
length(biomarkers2)
pars <- data.frame("trait"=biomarkers2)
pars
    
require('rstan')
require('rslurm')


[1] "Alanine_aminotransferase"   "Albumin"                   
 [3] "Alkaline_phosphatase"       "Apolipoprotein_A"          
 [5] "Apolipoprotein_B"           "Aspartate_aminotransferase"
 [7] "C_reactive_protein"         "Calcium"                   
 [9] "Cholesterol"                "Creatinine_in_urine"       
[11] "Creatinine"                 "Cystatin_C"                
[13] "Direct_bilirubin"           "eGFR"                      
[15] "Fasting_glucose"            "Gamma_glutamyltransferase" 
[17] "Glucose"                    "HDL_cholesterol"           
[19] "LDL_direct"                 "Lipoprotein_A"             
[21] "Microalbumin_in_urine"      "Non_albumin_protein"       
[23] "Oestradiol"                 "Phosphate"                 
[25] "Potassium_in_urine"         "Rheumatoid_factor"         
[27] "SHBG"                       "Sodium_in_urine"           
[29] "Testosterone"               "Total_bilirubin"           
[31] "Total_protein"              "Triglycerides"             
[33] "Urate"                      "Urea"                      
[35] "Vitamin_D"

[1] 35

trait                     
1  Alanine_aminotransferase  
2  Albumin                   
3  Alkaline_phosphatase      
4  Apolipoprotein_A          
5  Apolipoprotein_B          
6  Aspartate_aminotransferase
7  C_reactive_protein        
8  Calcium                   
9  Cholesterol               
10 Creatinine_in_urine       
11 Creatinine                
12 Cystatin_C                
13 Direct_bilirubin          
14 eGFR                      
15 Fasting_glucose           
16 Gamma_glutamyltransferase 
17 Glucose                   
18 HDL_cholesterol           
19 LDL_direct                
20 Lipoprotein_A             
21 Microalbumin_in_urine     
22 Non_albumin_protein       
23 Oestradiol                
24 Phosphate                 
25 Potassium_in_urine        
26 Rheumatoid_factor         
27 SHBG                      
28 Sodium_in_urine           
29 Testosterone              
30 Total_bilirubin           
31 Total_protein             
32 Triglycerides             
33 Urate                     
34 Urea                      
35 Vitamin_D

Loading required package: rstan
Loading required package: ggplot2
Loading required package: StanHeaders
rstan (Version 2.17.3, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)
Loading required package: rslurm


In [2]:
calcErrBarsHerit <- function(trait){
    
        
    
        # reformat into data frame

     my.df <- cbind(t(as.data.frame(res)), trait)
        my.df2 <- data.frame(cbind(my.df, rownames(my.df)))

    colnames(my.df2) <- c("hf", "hm", "hc", "trait", "int")
    my.df3 <- melt(my.df2, id.vars=c("trait", "int"), variable.name="sex")
         rownames(my.df3) <- NULL
	write.table(my.df3, file=sprintf("%s/h_err_%s_%s_%s.txt", DATA.FOLDER, trait, ndim, downsampled_str), row.names=FALSE, quote=FALSE, sep="\t")
    return(my.df3)
    }

# test it
t_bars <- calcErrBarsHerit("Testosterone")
head(t_bars)
# submit


Loading required package: MASS
Loading required package: Matrix
Loading required package: mnormt
Loading required package: qqman

For example usage please run: vignette('qqman')

Citation appreciated but not required:
Turner, S.D. qqman: an R package for visualizing GWAS results using Q-Q and manhattan plots. biorXiv DOI: 10.1101/005165 (2014).

Loading required package: reshape2
Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  1.4.2     ✔ purrr   0.2.4
✔ tidyr   0.8.1     ✔ dplyr   0.7.4
✔ readr   1.1.1     ✔ stringr 1.3.0
✔ tibble  1.4.2     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand()  masks Matrix::expand()
✖ tidyr::extract() masks rstan::extract()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ dplyr::select()  masks MASS::select()


ERROR: Error in names(x) <- value: 'names' attribute [5] must be the same length as the vector [4]


In [7]:

calcErrBarsHerit <- function(trait){

source("/scratch/PI/mrivas/users/erflynn/sex_div_gwas/mixture_model_scripts/model_utils.R")
    source("/scratch/PI/mrivas/users/erflynn/sex_div_gwas/mixture_model_scripts/heritability_utils.R")
    require('reshape2')
    require('tidyverse')
    ndim <- 2
    downsampled_str <- ""
    
    DATA.FOLDER <- "/scratch/PI/mrivas/users/erflynn/sex_div_gwas/data/biomarker"
    fit.file=sprintf("%s/f_%s.RData", DATA.FOLDER, trait)
    if (!file.exists(fit.file)){
            df <- data.frame(t(c("NA", trait, "NA", "NA")))
            colnames(df) <- c("value", "trait", "int", "sex")
            return(df)

        }
        load(file=fit.file)
        load(file=sprintf("%s/dat_%s.RData", DATA.FOLDER, trait))

        # extract all estimate
        list_of_draws <- rstan::extract(fit1)
        pi.draws <- list_of_draws$pi
        p <- pi.draws
        s.draws <- list_of_draws$Sigma
        Sigma <- s.draws

        # extract lower + upper pi
        ordered.p <- p[order(p[,2]),] # ordering p by the non-null component 
        p.lower <- ordered.p[0.025*nrow(ordered.p),]
        p.upper <- ordered.p[0.975*nrow(ordered.p),]
        p.center <- ordered.p[0.50*nrow(ordered.p),]
    
        # extract lower + upper sigma
        ordered.S <- Sigma[order(Sigma[,1,1]),,]
        s.upper <- ordered.S[0.975*dim(Sigma)[1],,]
        s.lower <- ordered.S[0.025*dim(Sigma)[1],,]
        s.center <- ordered.S[0.50*dim(Sigma)[1],,]

        # recalculate SNP membership
        dat2 <- dat
        dat2$categories <- NULL
        dat.u <- labelCategories(dat2, s.upper, p.upper)
        dat.l <- labelCategories(dat2, s.lower, p.lower)
        dat.c <- labelCategories(dat2, s.center, p.center)

        h.up <- overallHeritability(dat.u, s.upper, p.upper)
        h.low <- overallHeritability(dat.l, s.lower, p.lower)
        h.center <- overallHeritability(dat.c, s.center, p.center)

        res <- list("up"=h.up, "low"=h.low, "center"=h.center)
  
        # reformat into data frame

     my.df <- cbind(t(as.data.frame(res)), trait)
        my.df2 <- data.frame(cbind(my.df, rownames(my.df)))

    colnames(my.df2) <- c("hf", "hm", "trait", "int")
    my.df3 <- melt(my.df2, id.vars=c("trait", "int"), variable.name="sex")
         rownames(my.df3) <- NULL
	write.table(my.df3, file=sprintf("recalc_err/h_err_%s_%s_%s.txt", DATA.FOLDER, trait, ndim, downsampled_str), row.names=FALSE, quote=FALSE, sep="\t")
    return(my.df3)
    }

Warning message:
“attributes are not identical across measure variables; they will be dropped”

trait        int    sex value             
1 Testosterone up     hf  0.0572312988110804
2 Testosterone low    hf  0.0383525884464299
3 Testosterone center hf  0.0466389245093398
4 Testosterone up     hm  0.131494019580501 
5 Testosterone low    hm  0.0939680860750245
6 Testosterone center hm  0.117379897805179

In [9]:
sopt <- list(time = '1:00:00', memory='8000', partition='rbaltman')

sjob <- slurm_apply(calcErrBarsHerit, pars, nodes=10, jobname = 'err_bars_7', slurm_options=sopt , submit = TRUE )


In [10]:
ALT.DIR <- "../../data/1009/"
summary_files <- list.files(ALT.DIR, pattern="summary_(\\w)*_2_.txt")
alt_traits <- str_replace_all(summary_files, c("summary_dat_"="", "_2_.txt" =""))


pars2 <- data.frame("trait"=alt_traits)
pars2
calcErrBarsHerit2 <- function(trait){
    source("/scratch/PI/mrivas/users/erflynn/sex_div_gwas/mixture_model_scripts/model_utils.R")
    source("/scratch/PI/mrivas/users/erflynn/sex_div_gwas/mixture_model_scripts/heritability_utils.R")
    require('reshape2')
    ndim <- 2
    downsampled_str <- ""
    
    DATA.FOLDER <- "/scratch/PI/mrivas/users/erflynn/sex_div_gwas/data/1009"
    fit.file=sprintf("%s/f_%s.RData", DATA.FOLDER, trait)
    if (!file.exists(fit.file)){
            df <- data.frame(t(c("NA", trait, "NA", "NA")))
            colnames(df) <- c("value", "trait", "int", "sex")
            return(df)

        }
        load(file=fit.file)
        load(file=sprintf("%s/dat_%s.RData", DATA.FOLDER, trait))

        # extract all estimate
        list_of_draws <- rstan::extract(fit1)
        pi.draws <- list_of_draws$pi
        p <- pi.draws
        s.draws <- list_of_draws$Sigma
        Sigma <- s.draws

        # extract lower + upper pi
        ordered.p <- p[order(p[,2]),] # ordering p by the non-null component 
        p.lower <- ordered.p[0.025*nrow(ordered.p),]
        p.upper <- ordered.p[0.975*nrow(ordered.p),]
        p.center <- ordered.p[0.50*nrow(ordered.p),]
    
        # extract lower + upper sigma
        ordered.S <- Sigma[order(Sigma[,1,1]),,]
        s.upper <- ordered.S[0.975*dim(Sigma)[1],,]
        s.lower <- ordered.S[0.025*dim(Sigma)[1],,]
        s.center <- ordered.S[0.50*dim(Sigma)[1],,]

        # recalculate SNP membership
        dat2 <- dat
        dat2$categories <- NULL
        dat.u <- labelCategories(dat2, s.upper, p.upper)
        dat.l <- labelCategories(dat2, s.lower, p.lower)
        dat.c <- labelCategories(dat2, s.center, p.center)

        h.up <- overallHeritability(dat.u, s.upper, p.upper)
        h.low <- overallHeritability(dat.l, s.lower, p.lower)
        h.center <- overallHeritability(dat.c, s.center, p.center)

        res <- list("up"=h.up, "low"=h.low, "center"=h.center)

        # reformat into data frame

     my.df <- cbind(t(as.data.frame(res)), trait)
        my.df2 <- data.frame(cbind(my.df, rownames(my.df)))

    colnames(my.df2) <- c("hf", "hm", "trait", "int")
    my.df3 <- melt(my.df2, id.vars=c("trait", "int"), variable.name="sex")
         rownames(my.df3) <- NULL
	write.table(my.df3, file=sprintf("%s/h_err_%s_%s_%s.txt", DATA.FOLDER, trait, ndim, downsampled_str), row.names=FALSE, quote=FALSE, sep="\t")
    return(my.df3)
    }
sjob <- slurm_apply(calcErrBarsHerit2, pars2, nodes=10, jobname = 'err_bars_8', slurm_options=sopt , submit = TRUE )
 

trait   
1  arm_fp  
2  FEV_FVC 
3  INI20015
4  INI20021
5  INI20022
6  INI20150
7  INI21001
8  INI21021
9  INI23099
10 INI23100
11 INI23101
12 INI23102
13 INI23105
14 INI23106
15 INI23107
16 INI23108
17 INI23109
18 INI23110
19 INI23111
20 INI23112
21 INI23115
22 INI23116
23 INI23119
24 INI23120
25 INI23123
26 INI23124
27 INI23127
28 INI23128
29 INI30000
30 INI30010
⋮  ⋮       
46 INI30170
47 INI30180
48 INI30190
49 INI30200
50 INI30210
51 INI30220
52 INI30230
53 INI30240
54 INI30250
55 INI30260
56 INI30270
57 INI30280
58 INI30290
59 INI30300
60 INI30500
61 INI30510
62 INI30520
63 INI30530
64 INI3063 
65 INI3064 
66 INI4079 
67 INI4080 
68 INI46   
69 INI47   
70 INI48   
71 INI49   
72 INI50   
73 leg_fp  
74 trunk_fp
75 whr

In [4]:
trait <- "arm_fp"
    DATA.FOLDER <- "/scratch/PI/mrivas/users/erflynn/sex_div_gwas/data/1009"

  load(sprintf("%s/f_%s.RData", DATA.FOLDER, trait))
  load(sprintf("%s/dat_%s.RData", DATA.FOLDER, trait))

        # extract all estimate
        list_of_draws <- rstan::extract(fit1)
        pi.draws <- list_of_draws$pi
        p <- pi.draws
        s.draws <- list_of_draws$Sigma
        Sigma <- s.draws

        # extract lower + upper pi
        ordered.p <- p[order(p[,2]),] # ordering p by the non-null component 
        p.lower <- ordered.p[0.025*nrow(ordered.p),]
        p.upper <- ordered.p[0.975*nrow(ordered.p),]
        p.center <- ordered.p[0.50*nrow(ordered.p),]
    
        # extract lower + upper sigma
        ordered.S <- Sigma[order(Sigma[,1,1]),,]
        s.upper <- ordered.S[0.975*dim(Sigma)[1],,]
        s.lower <- ordered.S[0.025*dim(Sigma)[1],,]
        s.center <- ordered.S[0.50*dim(Sigma)[1],,]


In [14]:

p.lower
p.center
p.upper

s.lower
s.center
s.upper

print(fit1, pars=c("Sigma", "pi"), probs=c(0.025, 0.5, 0.975), digits_summary=7)

[1] 0.6040737 0.3959263

[1] 0.5786456 0.4213544

[1] 0.5487293 0.4512707

[,1]         [,2]        
[1,] 2.034593e-05 5.277175e-06
[2,] 5.277175e-06 1.094733e-05

[,1]         [,2]        
[1,] 2.185138e-05 6.283816e-06
[2,] 6.283816e-06 1.225395e-05

[,1]         [,2]        
[1,] 2.337962e-05 6.850301e-06
[2,] 6.850301e-06 1.285188e-05

Inference for Stan model: model1_no_loglik.
4 chains, each with iter=600; warmup=200; thin=1; 
post-warmup draws per chain=400, total post-warmup draws=1600.

                mean  se_mean        sd      2.5%       50%     97.5% n_eff
Sigma[1,1] 0.0000219 0.000000 0.0000008 0.0000203 0.0000219 0.0000234   651
Sigma[1,2] 0.0000061 0.000000 0.0000003 0.0000055 0.0000061 0.0000067   867
Sigma[2,1] 0.0000061 0.000000 0.0000003 0.0000055 0.0000061 0.0000067   867
Sigma[2,2] 0.0000118 0.000000 0.0000005 0.0000109 0.0000118 0.0000127   633
pi[1]      0.5781458 0.000567 0.0140299 0.5487278 0.5786439 0.6039722   612
pi[2]      0.4218542 0.000567 0.0140299 0.3960278 0.4213561 0.4512722   612
               Rhat
Sigma[1,1] 1.006971
Sigma[1,2] 1.000027
Sigma[2,1] 1.000027
Sigma[2,2] 1.004679
pi[1]      1.006033
pi[2]      1.006033

Samples were drawn using NUTS(diag_e) at Thu Oct 10 22:51:36 2019.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scal

In [8]:
source("/scratch/PI/mrivas/users/erflynn/sex_div_gwas/mixture_model_scripts/heritability_utils.R")
 source("/scratch/PI/mrivas/users/erflynn/sex_div_gwas/mixture_model_scripts/model_utils.R")

dat2 <- dat
        dat2$categories <- NULL
        dat.u <- labelCategories(dat2, s.upper, p.upper)
        dat.l <- labelCategories(dat2, s.lower, p.lower)
        dat.c <- labelCategories(dat2, s.center, p.center)

        h.up <- overallHeritability(dat.u, s.upper, p.upper)
        h.low <- overallHeritability(dat.l, s.lower, p.lower)
        h.center <- overallHeritability(dat.c, s.center, p.center)



Loading required package: MASS
Loading required package: Matrix
Loading required package: mnormt
Loading required package: qqman

For example usage please run: vignette('qqman')

Citation appreciated but not required:
Turner, S.D. qqman: an R package for visualizing GWAS results using Q-Q and manhattan plots. biorXiv DOI: 10.1101/005165 (2014).

Loading required package: rstan
Loading required package: ggplot2
Loading required package: StanHeaders
rstan (Version 2.17.3, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)


In [15]:

h.low
h.center
h.up

[1] 0.2578651 0.1705815

[1] 0.2592817 0.1775808

[1] 0.2586310 0.1742181

In [16]:

table(dat.l$categories)
table(dat.c$categories)
table(dat.u$categories)


     1      2 
177958   5592 


     1      2 
176639   6911 


     1      2 
174818   8732 

In [17]:
     h.up <- overallHeritability(dat.u, s.upper, p.upper)
        h.low <- overallHeritability(dat.l, s.lower, p.lower)
        h.center <- overallHeritability(dat.c, s.center, p.center)


In [18]:
h.up
h.center
h.low

[1] 0.2586310 0.1742181

[1] 0.2592817 0.1775808

[1] 0.2578651 0.1705815

In [23]:
se.p2 <- dat.u$dat$SE[dat.u$categories==2,]
n <- nrow(se.p2)
num_i <- n*(p.upper[2])*s.upper[1,1]
    
h_i <- num_i/(num_i + sum(se.p2[,1]))
sum(se.p2[,1])
num_i
h_i
num_i <- n*(p.upper[2])*s.upper[2,2]
    
h_2 <- num_i/(num_i + sum(se.p2[,2]))
sum(se.p2[,2])
num_i
h_2

[1] 0.2640841

[1] 0.09212732

[1] 0.258631

[1] 0.2400433

[1] 0.0506428

[1] 0.1742181

In [27]:
se.p2 <- dat.u$dat$SE[dat.c$categories==2,]
n <- nrow(se.p2)
num_i <- n*(p.center[2])*s.center[1,1]
    
h_i <- num_i/(num_i + sum(se.p2[,1]))
sum(se.p2[,1])
num_i
h_i
num_i <- n*(p.center[2])*s.center[2,2]
    
h_2 <- num_i/(num_i + sum(se.p2[,2]))
sum(se.p2[,2])
num_i
h_2

[1] 0.181781

[1] 0.0636308

[1] 0.2592817

[1] 0.1652578

[1] 0.03568326

[1] 0.1775808

[1] 0.2571429

[1] 0.2601626

In [ ]:


# LD FILTER SNPS AGAIN

In [3]:
require('tidyverse')

snps.to.keep <- read.table("../../data/snp_filt_list2.txt", header=FALSE, 
    colClasses="character")


In [4]:
f_snps <- read_csv("../../data/biomarker/m2/all_f_df.txt")
m_snps <- read_csv("../../data/biomarker/m2/all_m_df.txt")
head(f_snps)

Parsed with column specification:
cols(
  SNP = col_character(),
  CHR = col_integer(),
  BP = col_integer(),
  B_f = col_double(),
  B_m = col_double(),
  SE_f = col_double(),
  SE_m = col_double(),
  p_m = col_double(),
  p_f = col_double(),
  p1 = col_double(),
  p2 = col_double(),
  p3 = col_double(),
  p4 = col_double(),
  gene = col_character(),
  trait = col_character()
)
Parsed with column specification:
cols(
  SNP = col_character(),
  CHR = col_integer(),
  BP = col_integer(),
  B_f = col_double(),
  B_m = col_double(),
  SE_f = col_double(),
  SE_m = col_double(),
  p_m = col_double(),
  p_f = col_double(),
  p1 = col_double(),
  p2 = col_double(),
  p3 = col_double(),
  p4 = col_double(),
  gene = col_character(),
  trait = col_character()
)
Warning message:
“2 parsing failures.
row # A tibble: 2 x 5 col     row col   expected   actual     file                                   expected   <int> <chr> <chr>      <chr>      <chr>                                  actual 1   17

SNP           CHR BP        B_f       B_m       SE_f      SE_m     
1 rs121918002    1   21900176 -2.469460 -2.380950 0.1289750 0.1427530
2 Affx-52354612 15   58838103  0.602026  0.500896 0.0884069 0.0822425
3 Affx-89019160 10  101157378 -2.673440 -2.460210 0.0864675 0.0846102
4 rs146049867   10  101165952 -0.697329 -0.477447 0.0636471 0.0625425
5 rs8177505      6  160679656  0.510406  0.485647 0.0493749 0.0514179
6 Affx-92045743 20   23615951 -3.353020 -3.380690 0.1403860 0.1550710
  p_m          p_f          p1            p2        p3           p4           
1  2.13323e-62  1.29078e-81  1.046546e-73 1.0000000 2.368780e-19  1.607599e-54
2  1.12875e-09  9.82022e-12  1.959040e-04 0.5591625 1.546044e-02  4.251811e-01
3 2.36206e-185 3.18713e-209  0.000000e+00 1.0000000 3.559281e-25 1.015780e-106
4  2.29014e-14  6.36401e-28  2.918874e-20 1.0000000 8.670623e-15  4.160020e-08
5  3.59985e-21  4.87012e-25  2.383065e-17 0.9996841 2.179201e-04  9.801127e-05
6 3.51477e-105 8.23929e-126 8.231059e-118 1.0000000 4.192534e-22 2.095239e-100
  gene    trait                     
1 ALPL    Alkaline_phosphatase      
2 LIPC    Apolipoprotein_A          
3 GOT1    Aspartate_aminotransferase
4 GOT1    Aspartate_aminotransferase
5 SLC22A2 Creatinine                
6 CST3    Cystatin_C

In [6]:
length(f_snps$SNP)
length(intersect(f_snps$SNP, snps.to.keep$V1))

length(m_snps$SNP)
length(intersect(m_snps$SNP, snps.to.keep$V1))

[1] 115

[1] 30

[1] 504

[1] 112

In [11]:
filter(m_snps, SNP %in% snps.to.keep$V1) %>% arrange(trait, CHR, BP) %>% write_csv("../../data/aggreg_results_1011/m2_m_only_ld.txt")

In [10]:
filter(f_snps, SNP %in% snps.to.keep$V1) %>% arrange(trait, CHR, BP )%>% write_csv("../../data/aggreg_results_1011/m2_f_only_ld.txt")